In [3]:
import numpy
from matplotlib import pyplot as plt
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils

In [6]:
##fix the seed for reproducability
seed = 7
numpy.random.seed(seed)


In [9]:
## load the data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

##Normalize the data(0-255 to 0.0-1.0)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255.0
X_test = X_test/255.0

##one hot encoding for the output classes
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
nclasses = y_test.shape[1]

In [14]:
## define your model
model = Sequential()
model.add(Convolution2D(32,3,3, input_shape=(3,32,32), activation='relu', border_mode='same'))
model.add(Dropout(0.2))
model.add(Convolution2D(32,3,3, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64,3,3, activation='relu', border_mode='same'))
model.add(Dropout(0.2))
model.add(Convolution2D(64,3,3, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Convolution2D(128,3,3, activation='relu', border_mode='same'))
model.add(Dropout(0.2))
model.add(Convolution2D(128,3,3, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(1024, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))

model.add(Dense(nclasses,activation='softmax'))



In [17]:
#compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum = 0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
#fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=epochs, batch_size=64)
score = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy : %.2f%% " %(score[1]*100))